In [2]:
import numpy as np

# network size N
N = 5

# number of edges
M = N*3

## initialize time
T = 0
## choose Tmax
Tmax = 5

# network size N
N = 5

# number of edges
M = N*3

Tmax = 5


In [48]:
class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val

def find_tranformation_matrix(b, E):

    ## find the number of events that occur in the interval from T = 0 to T = Tmax
    n = np.random.poisson(lam = Tmax*(N+b*M))

    ## find times for all events
    times = np.sort(np.random.uniform(0, Tmax, n))

    ## initialize boolean identity Matrix A of dimensions NxN
    X = np.identity(N, dtype=bool)
    
    vf = np.vectorize(lambda x: 2**x)
    ints = np.array(list(range(N)))
    I = vf(ints)[::-1]

    counts = Counter()
    for element in I:
        counts.increment(element, 1)

    distinct_rows = N

    # run simulation
    for t in range(n):
        
        if np.random.random() < N/(N + b*M):

            i = np.random.randint(N)
        
            row_i_not_distinct = I[i] in counts and counts[I[i]] > 1

            if I[i] != 0:

                if 0 in counts and counts[0] >= 1:

                    if row_i_not_distinct == False:

                        distinct_rows -= 1

                     
                else:

                    if row_i_not_distinct == True:
                            
                        distinct_rows = min(distinct_rows + 1, N)
                
                counts.increment(I[i], -1)

                X[i, :] = False

                I[i] = 0

                counts.increment(0, 1)
        else:

            i,j = E[np.random.randint(M)]

            if (I[i] != 0 or I[j] != 0) and I[i] != I[j]:
                    
                new = int(''.join('1' if x else '0' for x in np.logical_or(X[i], X[j])), 2)
                counts.increment(I[i], -1)
                counts.increment(I[j], -1)
                
                new_not_distinct = new in counts and counts[new] >= 1
                    
                row_i_not_distinct = I[i] in counts and counts[I[i]] >= 1

                row_j_not_distinct = I[j] in counts and counts[I[j]] >= 1

                counts.increment(new, 2)
                I[i] = I[j] = new

                if new_not_distinct == False:
                    if row_i_not_distinct == False and row_j_not_distinct == False:
                        distinct_rows -= 1
                    if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                        distinct_rows = distinct_rows
                    if row_i_not_distinct == True and row_j_not_distinct == True:
                        distinct_rows = min(distinct_rows + 1, N)

                else:
                    if row_i_not_distinct == False and row_j_not_distinct == False:
                        distinct_rows -= 2
                    if (row_i_not_distinct == False and row_j_not_distinct == True) or (row_i_not_distinct == True and row_j_not_distinct == False):
                        distinct_rows -= 1

                X[i] = X[j] = np.logical_or(X[i], X[j])
                
        distinct_rows_check = np.unique(X, axis=0).shape[0]
        assert distinct_rows == distinct_rows_check, f"distinct rows is {distinct_rows} but should be {distinct_rows_check}"
        ## break the simulation because the pseudo mixing time has been found (the number of distinct rows and thus distinct columns is 2)

        if distinct_rows == 2:
            
            break

    return X, I


In [49]:
b = 0.5

## random edge matrix
E = np.random.randint(0,N,size=2*M).reshape((N*3, 2))

x0 = np.ones(N, dtype=bool)
matrix_A = find_tranformation_matrix(b, E)
print(matrix_A)


(array([[False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False, False, False],
       [False, False, False,  True,  True],
       [False, False, False,  True,  True]]), array([0, 0, 0, 3, 3]))


In [9]:
X = np.array([[False, False,  True,  True, False],
       [False,  True, False, False,  True],
       [False, False,  True,  True, False],
       [False, False,  True,  True, False],
       [False,  True, False, False,  True]])
print(X)
print(int(''.join('1' if x else '0' for x in X[0]), 2))
print(int(''.join('1' if x else '0' for x in X[3]), 2))
int(''.join('1' if x else '0' for x in np.logical_or(X[0], X[1])), 2)


[[False False  True  True False]
 [False  True False False  True]
 [False False  True  True False]
 [False False  True  True False]
 [False  True False False  True]]
6
6


15

In [30]:
X = np.array([[True, False,  True,  True, False],
       [False,  True, False, False,  True],
       [False, False,  True,  False, False],
       [False, False,  False,  True, False],
       [False,  True, False, False,  True]])

I = []
for i in range(X.shape[0]):
       I.append(int(''.join('1' if x else '0' for x in X[i]), 2))

print(I)

counts = Counter()
for element in I:
       counts.increment(element, 1)
print(counts)

if (I[1] != 0 or I[0] != 0) and I[1] != I[0]:

    new = int(''.join('1' if x else '0' for x in np.logical_or(X[1], X[3])), 2)
    counts.increment(I[1], -1)
    counts.increment(I[3], -1)
    new_not_distinct = new in counts and counts[new] >= 1
    print(new_not_distinct)

    print(counts)
    row_i_not_distinct = I[1] in counts and counts[I[1]] >= 1
    print(row_i_not_distinct)

    row_j_not_distinct = I[3] in counts and counts[I[3]] >= 1
    print(row_j_not_distinct)

    counts.increment(new, 1)
    I[3] = I[1] = new


[22, 9, 4, 2, 9]
{22: 1, 4: 1, 2: 1, 9: 2}
False
{22: 1, 4: 1, 9: 1}
True
False


In [5]:
## initialize boolean identity Matrix A of dimensions NxN
X = np.identity(N, dtype=bool)
    
vf = np.vectorize(lambda x: 2**x)
ints = np.array(list(range(N)))
I = vf(ints)[::-1]

counts = Counter()
for element in I:
    counts.increment(element, 1)

print(X)
print(I)
print(counts)


# new = int(''.join('1' if x else '0' for x in np.logical_or(X[i], X[j])), 2)

print(int('10000'[::-1], 2))


[[ True False False False False]
 [False  True False False False]
 [False False  True False False]
 [False False False  True False]
 [False False False False  True]]
[16  8  4  2  1]
{16: 1, 8: 1, 4: 1, 2: 1, 1: 1}
1


In [4]:
class Counter(dict):
    def increment(self, item, delta = 1):
        new_val = delta + self.pop(item, 0)
        if new_val > 0:
            self[item] = new_val


vf = np.vectorize(lambda x: 2**x)
ints = np.array(list(range(N)))
I = vf(ints)

counts = Counter()
for element in I:
    counts.increment(element, 1)

counts.increment(1, 1)
print(len(counts))
print(counts)
print(counts[1])

row_i_not_distinct = I[1] in counts and counts[I[1]] > 1
print(row_i_not_distinct)

0 in counts and counts[0] >= 1


5
{2: 1, 4: 1, 8: 1, 16: 1, 1: 2}
2
False


False

In [21]:
I = [1, 2, 0, 0, 0]

if (I[1] != 0 or I[0] != 0) and I[1] != I[0]:
    print('yes')


yes


In [2]:
# 5 | 7

0 or 7


7